In [65]:
# Imports used in this notebook.
import matplotlib as mpl
import ee
import folium
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive
from pprint import pprint as pp
from uuid import uuid1 as uniqueID

# Must authenticate your EE account before use of the package.
ee.Authenticate()
#ee.Initialize()
ee.Initialize(project='ksu-skytruth-vpca-automation', opt_url='https://earthengine-highvolume.googleapis.com')



In [66]:
import geemap
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.


  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  # water_pixels = (image.select('QA60').eq(0).
  qa = image.select('MSK_CLDPRB').eq(0).And(image.select('SCL').eq(4))

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).divide(10000)

dateRange = ['2023-04-26', '2023-04-27']


# upperLeft = [-81.1583124,  41.0254715 ]
# lowerRight = [-81.1469973,  41.0135215 ]

# roi = ee.Geometry.MultiPolygon( [
#      [[-81.23416, 41.19271],
# [-81.23435, 41.19436],
# [-81.23523, 41.19404],
# [-81.23572, 41.19408],
# [-81.23639, 41.19368],
# [-81.23671, 41.19328],
# [-81.23677, 41.19293],
# [-81.23802, 41.19258],
# [-81.23582, 41.19261],
# [-81.23454, 41.19261],
# [-81.23413, 41.19271]],
# [ [ -81.227544, 41.198116],
# [-81.2274373, 41.20185],
# [-81.229058, 41.201828],
# [-81.22917, 41.201263],
# [-81.228915, 41.20119],
# [-81.229034, 41.198147]]
#      ])

     # #JD
roi = ee.Geometry.Polygon( [
  [-80.5894, 40.86986],
[-80.59206, 40.86972],
 [-80.59366, 40.86776],
 [-80.59369, 40.86487],
 [-80.59068, 40.86538],
[-80.59064, 40.86401],
 [-80.58983, 40.86396]


])



# roi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1])
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2024-05-08', '2024-05-09').filterBounds(roi)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))
    .map(mask_s2_clouds)
)


#  def get_water(image):

#     # Water with cloud check, but note this is the not bit addition method.
#     water_pixels = (image.select('QA60').eq(0).And(image.select('SCL').eq(6)))
#     # water_pixels=image.select('QA60')
#     # cloudBitMask=1 << 10
#     # cirrusBitMask=1 << 11
#     # mask = water_pixels.bitwiseAnd(cloudBitMask).eq(0).And(water_pixels.bitwiseAnd(cirrusBitMask).eq(0))
#     # water_pixels = (image.select('SCL').eq(6).And(image.select('SCL').lt(7)))
#     # Mask all non-water pixels in the image.
#     image = image.updateMask(water_pixels)

#     return image

  # Water with cloud check, but note this is not bit the addition method.

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
# m.addLayer(roi, {}, 'ROI')



# prompt: code to generate the center of the roi and show the map

# Get the centroid of the ROI.
centroid = roi.centroid()

# Print the centroid coordinates.
print('Centroid coordinates:', centroid.getInfo()['coordinates'])

# Center the map on the centroid.
m.centerObject(centroid, 15)

# m.set_center(-81.1583124, 41.0254715, 15)
m.add_layer(dataset, visualization, 'RGB')
# style = {'fillColor': '00000000', 'color': '000000', 'weight': 2}
# m.addLayer(roi, style, 'ROI outline')

m

Centroid coordinates: [-81.23547695953847, 41.1932769357655]


Map(center=[41.1932769357655, -81.23547695953847], controls=(WidgetControl(options=['position', 'transparent_b…

In [67]:
link = dataset.mean().getDownloadURL({
    'scale': 10,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': roi})
print(link)

https://earthengine-highvolume.googleapis.com/v1/projects/ksu-skytruth-vpca-automation/thumbnails/8f2c40ed086adaee7b8b035fa163efea-4ebf79d39fd9f38e696c96dd0ef25171:getPixels
